In [1]:
""" My first attempt looking at this data in the context of graphical models """


import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import pgmpy

import warnings
warnings.filterwarnings("ignore")

sns.set(style="ticks")
sns.set_context(context="talk")

In [2]:
infile = "/Users/kmcmanus/Documents/classes/digitalhealth_project/data/formatted_data/20200628_sleep_pos_5S_cleaned.csv"
df = pd.read_csv(infile, index_col='datetime', parse_dates=True, infer_datetime_format=True)
df["sleep_night"] = pd.to_datetime(df["sleep_night"])
df = df.dropna()
print("Total # rows: {}".format(df.shape[0]))
df.head()

Total # rows: 100818


,Orientation,Inclination,sleep_night,SpO2(%),Pulse Rate(bpm),Motion,Vibration,ODI,orient_bin,low_oxygen,hour,complete_hour,complete_night
datetime,,,,,,,,,,,,,
2020-04-21 20:59:00,-81.72,68.09,2020-04-21,96.0,65.0,1.0,0.0,0.0,1.0,0,20,0,1
2020-04-21 20:59:05,-82.20,68.04,2020-04-21,96.0,64.0,1.0,0.0,0.0,1.0,0,20,0,1
2020-04-21 20:59:10,-76.71,71.69,2020-04-21,96.0,64.0,21.0,0.0,0.0,1.0,0,20,0,1
2020-04-21 20:59:15,-80.81,69.74,2020-04-21,96.0,64.0,18.0,0.0,0.0,1.0,0,20,0,1
2020-04-21 20:59:20,-79.80,70.00,2020-04-21,94.0,70.0,0.0,0.0,0.0,1.0,0,20,0,1


In [7]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.estimators import BayesianEstimator

model = BayesianModel([('orient_bin', 'ODI'), ('hour', 'ODI'), ('Pulse Rate(bpm)', 'ODI')])  # fruit -> tasty <- size

from pgmpy.estimators import ParameterEstimator # Literally had to go in an manually change this file
from pgmpy.estimators import BDeuScore 
pe = ParameterEstimator(model, df)
print("\n", pe.state_counts('orient_bin'))  # unconditional
print("\n", pe.state_counts('hour'))  # conditional on fruit and size
print("\n", pe.state_counts('Pulse Rate(bpm)'))  # conditional on fruit and size
print("\n", pe.state_counts('ODI'))  # conditional on fruit and size


       orient_bin
-1.0       28653
 0.0       29534
 1.0       42631

      hour
0   10014
1   10104
2   11159
3   10690
4    9906
5    8922
6    1892
20    813
21  13114
22  12979
23  11225

       Pulse Rate(bpm)
47.0                1
50.0                1
51.0               31
52.0              116
53.0              592
54.0             2270
55.0             4540
56.0             6876
57.0             7631
58.0             8074
59.0             8908
60.0             9046
61.0             8799
62.0             7355
63.0             6810
64.0             6843
65.0             5490
66.0             4692
67.0             3387
68.0             2921
69.0             2094
70.0             1384
71.0              844
72.0              563
73.0              346
74.0              200
75.0              194
76.0              120
77.0              111
78.0              125
79.0               74
80.0               56
81.0               49
82.0               44
83.0               35
84.0          

In [8]:
model.fit(df, estimator=BayesianEstimator, prior_type="BDeu") # default equivalent_sample_size=5
for cpd in model.get_cpds():
    print(cpd)

+------------------+----------+
| orient_bin(-1.0) | 0.284208 |
+------------------+----------+
| orient_bin(0.0)  | 0.292946 |
+------------------+----------+
| orient_bin(1.0)  | 0.422847 |
+------------------+----------+
+-----------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-------------

In [ ]:
# Ok what do I do with this?

